This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [31]:
import os

os.environ["OPENAI_API_KEY"] = "voc-30237581412667733285796716cdfcba5865.61612652"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.chat_models import init_chat_model
from langchain.prompts import ChatPromptTemplate


1. Generate Listings

In [32]:


homes_gen_template = """
Generate CSV formatted descriptions for home listings. There should be one CSV header row at the top row, and then precisely {home_rows} home rows, to make a total of precisely {total_rows} rows in the CSV result.  
The home descriptions should be interesting but also realistic, and display a broad range of homes catering to many different house purchaser tastes.
Be creative and imaginative.  

The CSV format is essential, and you must ensure all prices are between single quotes. 

Always include the following Columns in the CSV:
- Neighborhood
- Price
- Bedrooms number
- Bathrooms number
- House Size in square feet
- Detailed Description
- Detailed Neighborhood Description


EXAMPLE with precisely 2 home rows and 3 total rows in the CSV:
Neighborhood,Price,Bedrooms,Bathrooms,House Size (sqft),Description,Neighborhood Description
Sunnyvale,'650,000',4,3,'2,500',"Discover your dream home in Sunnyvale! This stunning 4-bedroom, 3-bathroom residence features an expansive open floor plan, perfect for entertaining. The gourmet kitchen is equipped with stainless steel appliances and a large island, while the spacious living room boasts a cozy fireplace. Step outside to a beautifully landscaped backyard, ideal for summer barbecues and family gatherings. The master suite offers a private retreat with a luxurious en-suite bathroom and walk-in closet. Don't miss this opportunity to own a piece of paradise in Sunnyvale!","Sunnyvale is a vibrant community known for its excellent schools and family-friendly atmosphere. With numerous parks, playgrounds, and recreational facilities, there's always something to do. The neighborhood hosts seasonal farmers' markets and community events, fostering a strong sense of belonging. Enjoy easy access to shopping, dining, and major highways, making it a convenient location for commuters."
Maplewood,'1,200,000',5,4,'3,800',"Welcome to this exquisite 5-bedroom, 4-bathroom estate in the prestigious Maplewood neighborhood. This luxurious home features high ceilings, elegant crown molding, and hardwood floors throughout. The chef's kitchen is a culinary delight, complete with granite countertops and a spacious pantry. The expansive master suite includes a spa-like bathroom and a private balcony overlooking the lush backyard. Enjoy movie nights in the dedicated home theater or host gatherings in the formal dining room. This home is a true masterpiece!","Maplewood is an upscale neighborhood characterized by tree-lined streets and beautifully manicured lawns. Residents enjoy a peaceful suburban lifestyle while being just minutes away from fine dining, boutique shopping, and cultural attractions. The community is known for its top-rated schools and active neighborhood associations, which organize events and activities throughout the year. With easy access to public transportation and major highways, Maplewood offers the perfect blend of luxury and convenience."
"""

prompt = ChatPromptTemplate.from_template(homes_gen_template)


In [33]:

llm = init_chat_model("gpt-4o", model_provider="openai", temperature=0.0,max_tokens=12000)

In [34]:
chain = prompt | llm

In [46]:

home_rows = 12

total_rows = home_rows + 1

result = chain.invoke(
    {"home_rows": home_rows,
     "total_rows": total_rows}
)
print(result)

content='Neighborhood,Price,Bedrooms,Bathrooms,House Size (sqft),Description,Neighborhood Description\nGreenfield,\'450,000\',3,2,\'1,800\',"Charming 3-bedroom, 2-bathroom bungalow in the heart of Greenfield. This cozy home features a sunlit living room with hardwood floors and a vintage fireplace. The updated kitchen boasts quartz countertops and modern appliances. Enjoy the serene backyard with a garden and patio, perfect for relaxing evenings. The master bedroom offers a peaceful retreat with ample closet space. A perfect blend of classic charm and modern convenience awaits you in Greenfield!","Greenfield is a quaint neighborhood known for its friendly community and lush green spaces. With several parks and walking trails, it\'s a haven for outdoor enthusiasts. The area hosts regular community events, including art fairs and outdoor concerts. Greenfield\'s local shops and cafes offer a unique, small-town feel while being conveniently close to urban amenities."\nRiverside,\'850,000\'

In [47]:
with open("home_listings.csv", "w") as file:
    content = result.content
    # Simple extraction between markers
    if "```csv" in content and "```" in content:
        start = content.find("```csv") + 6  # Skip past "```csv"
        end = content.rfind("```")
        csv_content = content[start:end].strip()
        file.write(csv_content)
    else:
        file.write(content)